In [1]:
import numpy as np
import pandas as pd
import recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#df_business=pd.read_feather("../data/business.feather")
#df_review=pd.read_feather("../data/review.feather",columns=['business_id','user_id'])
df_review=pd.read_csv("../data/review.csv",usecols=['business_id','user_id'])

In [3]:
#print(df_business.head())
df_review.head()

,user_id,business_id
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg
1,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ
2,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw
3,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA
4,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ


In [4]:
rec.mostpopular(df_review,10)

,business_id,score
0,4JNXUYY8wbaaDmk3BPzlWw,8570
1,RESDUcs7fIiihp38-d6_6g,8568
2,K7lWdNUhCbcnEvI0NhGewg,6887
3,f4x1YBxkLrZg652xt2KR5g,5847
4,cYwJA2A6I12KNkm2rtXd5g,5575
5,DkYS3arLOhA8si5uUEmHOw,5206
6,2weQS-RnoOBhb1KsHKyoSQ,4534
7,5LNZ67Yw9RD6nf4_UhXOjw,4522
8,iCQpiavjjPzJ5_3gPD5Ebg,4351
9,SMPbvZLSMMb7KU76YNYMGg,4350


In [5]:
df_review=None
tmp_rec_list=_

In [6]:

df_business=pd.read_csv("../data/business.csv",usecols=['business_id','categories'])
tmp_rec_list=pd.merge(tmp_rec_list,df_business,left_on='business_id',right_on='business_id').drop_duplicates().reset_index(drop=True)
df_business=None
tmp_rec_list

,business_id,score,categories
0,4JNXUYY8wbaaDmk3BPzlWw,8570,"['Food', 'French', 'Breakfast & Brunch', 'Rest..."
1,RESDUcs7fIiihp38-d6_6g,8568,"['Restaurants', 'Buffets', 'Sandwiches', 'Food..."
2,K7lWdNUhCbcnEvI0NhGewg,6887,"['Buffets', 'Breakfast & Brunch', 'Restaurants']"
3,f4x1YBxkLrZg652xt2KR5g,5847,"['Breakfast & Brunch', 'Food', 'Restaurants', ..."
4,cYwJA2A6I12KNkm2rtXd5g,5575,"['Restaurants', 'Burgers', 'American (Traditio..."
5,DkYS3arLOhA8si5uUEmHOw,5206,"['Food Delivery Services', 'Salad', 'Sandwiche..."
6,2weQS-RnoOBhb1KsHKyoSQ,4534,"['Beauty & Spas', 'Food', 'Event Planning & Se..."
7,5LNZ67Yw9RD6nf4_UhXOjw,4522,"['Casinos', 'Hotels', 'Event Planning & Servic..."
8,iCQpiavjjPzJ5_3gPD5Ebg,4351,"['Pizza', 'Restaurants']"
9,SMPbvZLSMMb7KU76YNYMGg,4350,"['Hotels & Travel', 'Arts & Entertainment', 'H..."


# Categorias relevantes

In [7]:
Lambda = 0.5
Beta = 0.5

In [8]:
df_categories=pd.read_json("../data/categories.json")

# dicionário alias title 2 way
dict_alias_title={}
for index, row in df_categories.iterrows():
    dict_alias_title[row['alias']]=row['title']
    dict_alias_title[row['title']]=row['alias']


category_tree= nx.DiGraph()
for index, row in df_categories.iterrows():
    if not row['parents']:
        category_tree.add_edge(row['alias'],'root') # root node if no parents
    else:
        for parent_label in row['parents']:
            category_tree.add_edge(row['alias'],parent_label)

undirected_category_tree=category_tree.to_undirected()
df_categories=None

In [9]:
nx.shortest_path_length(undirected_category_tree,'aircraftdealers','airsoft')

4

In [15]:
# lista temporária e lista final
# rec_list=pd.DataFrame(columns=['business_id','score'])

# k=len(tmp_rec_list) ## size of rec list
# for i in range(k):
#     print(i)
#     for index,row in tmp_rec_list.iterrows():
#         business_id=row['business_id']
#         score=row['score']
#         print(index,business_id,score)


In [10]:
eval(tmp_rec_list.iloc[0]['categories'])

['Food', 'French', 'Breakfast & Brunch', 'Restaurants', 'Steakhouses']

In [25]:

        
def category_dis_sim(category1,category2):
    dissim=0.0
    spd=nx.shortest_path_length(undirected_category_tree,category1,category2)
    sim = 1.0 / (1.0 + spd)
    dissim=1.0-sim
    return dissim
    

def min_dist_to_list_cat(business,rec_list):
    min_dissim=1.0
    rec_list_size=len(rec_list)
    if rec_list_size==0:
        min_dissim=1.0
    else:
        for index,row in rec_list.iterrows():
            local_min_distance=1
            cur_distance=0
            for category1 in eval(business['categories']):
                for category2 in eval(row['categories']):
                    cur_distance=category_dis_sim(
                        dict_alias_title[category1],
                        dict_alias_title[category2])
                    #print(category1,category2,cur_distance,local_min_distance)
                    local_min_distance=min(local_min_distance,cur_distance)
            min_dissim=min(min_dissim,local_min_distance)
    return min_dissim
        
def objective_ild(business,rec_list):
    # /*(1-_cfg._divWeight) * relevance + */_cfg._divWeight * diversity; where _cfg._divWeight = 0.9
    ret_val= 0.0
    diversity=0.0
    div_weight=0.9
    diversity=min_dist_to_list_cat(business,rec_list)
    ret_val = div_weight*diversity #+ (1-div_weight)*relevance
    return ret_val

min_dist_to_list_cat(tmp_rec_list.iloc[0],tmp_rec_list.iloc[1:9])

0.0

In [ ]:
# for index,row in tmp_rec_list.iterrows():
#     business_id=row['business_id']
#     score=row['score']
#     print(index,business_id,score)

In [24]:
tmp_rec_list.iloc[0]['categories']+ tmp_rec_list.iloc[1]['categories']

"['Food', 'French', 'Breakfast & Brunch', 'Restaurants', 'Steakhouses']['Restaurants', 'Buffets', 'Sandwiches', 'Food', 'Breakfast & Brunch']"